**Coursebook: Exploratory Data Analysis**
- Part 2 of Data Analytics Specialization
- Course Length: 12 hours
- Last Updated: April 2020

___

- Author: [Samuel Chan](https://github.com/onlyphantom)
- Developed by [Algoritma](https://algorit.ma)'s product division and instructors team

# Background

## Top-Down Approach 

The coursebook is part of the **Data Analytics Specialization** offered by [Algoritma](https://algorit.ma). It takes a more accessible approach compared to Algoritma's core educational products, by getting participants to overcome the "how" barrier first, rather than a detailed breakdown of the "why". 

This translates to an overall easier learning curve, one where the reader is prompted to write short snippets of code in frequent intervals, before being offered an explanation on the underlying theoretical frameworks. Instead of mastering the syntactic design of the Python programming language, then moving into data structures, and then the `pandas` library, and then the mathematical details in an imputation algorithm, and its code implementation; we would do the opposite: Implement the imputation, then a succinct explanation of why it works and applicational considerations (what to look out for, what are assumptions it made, when _not_ to use it etc).

## Training Objectives

This coursebook is intended for participants who have completed the preceding courses offered in the **Data Analytics Developer** Specialization. This is the second course, **Exploratory Data Analysis**

The coursebook focuses on:
- Why and What: Exploratory Data Analysis
- Date Time objects
- Categorical data types
- Cross Tabulation and Pivot Table
- Treating Duplicates and Missing Values 

At the end of this course is a Learn-by-Building section, where you are expected to apply all that you've learned on a new dataset, and attempt the given questions.

# Data Preparation and Exploration

About 60 years ago, John Tukey defined data analysis as the "procedures for analyzing data, techniques for interpreting the results of such procedures ... and all the machinery of mathematical statistics which apply to analyzing data". His championing of EDA encouraged the development of statsitical computing packages, especially S at Bell Labs (which later inspired R).

He wrote a book titled _Exploratory Data Analysis_ arguing that too much emphasis in statistics was placed on hypothesis testing (confirmatory data analysis) while not enough was placed on the discovery of the unexpected. 

> Exploratory data analysis isolates patterns and features of the data and reveals these forcefully to the analyst.

This course aims to present a selection of EDA techniques -- some developed by John Tukey himself -- but with a special emphasis on its application to modern business analytics.

In the previous course, we've got our hands on a few common techniques:

- `.head()` and `.tail()`
- `.describe()`
- `.shape` and `.size`
- `.axes`
- `.dtypes`

In the following chapters, we'll expand our EDA toolset with the following additions:  

- Tables
- Cross-Tables and Aggregates
- Using `aggfunc` for aggregate functions
- Pivot Tables

In [1]:
import pandas as pd
import numpy as np
print(pd.__version__)

1.0.3


In [59]:
# pandas output display setup

pd.set_option('display.float_format', lambda x: '%.2f' % x) 
#pd.options.display.float_format = '{:,}'.format

## Working with Datetime

Given the program's special emphasis on business-driven analytics, one data type of particular interest to us is the `datetime`. In the first part of this coursebook, we've seen an example of `datetime` in the section introducing data types (`employees.joined`).

A large portion of data science work performed by business executives involve time series and/or dates (think about the kind of data science work done by computer vision researchers, and compare that to the work done by credit rating analysts or marketing executives and this special relationship between business and datetime data becomes apparent), so adding a level of familiarity with this format will serve you well in the long run. 

As a start, let's read our data,`household.csv`:

In [106]:
household = pd.read_csv("data_input/household.csv")
household.head()

,receipt_id,receipts_item_id,purchase_time,category,sub_category,format,unit_price,discount,quantity,yearmonth
0,9622257,32369294,7/22/2018 21:19,Rice,Rice,supermarket,128000.00,0,1,2018-07
1,9446359,31885876,7/15/2018 16:17,Rice,Rice,minimarket,102750.00,0,1,2018-07
2,9470290,31930241,7/15/2018 12:12,Rice,Rice,supermarket,64000.00,0,3,2018-07
3,9643416,32418582,7/24/2018 8:27,Rice,Rice,minimarket,65000.00,0,1,2018-07
4,9692093,32561236,7/26/2018 11:28,Rice,Rice,supermarket,124500.00,0,1,2018-07


In [61]:
household.dtypes

receipt_id            int64
receipts_item_id      int64
purchase_time        object
category             object
sub_category         object
format               object
unit_price          float64
discount              int64
quantity              int64
yearmonth            object
dtype: object

---

1. using `parse_dates` in `pd.read_csv()`: digunakan ketika kita sudah mengetahui ada informasi datetime pada data yang akan dibaca (sudah terbiasa dengan datanya)

In [62]:
df = pd.read_csv("data_input/household.csv", parse_dates=['purchase_time'])
df.head()

,receipt_id,receipts_item_id,purchase_time,category,sub_category,format,unit_price,discount,quantity,yearmonth
0,9622257,32369294,2018-07-22 21:19:00,Rice,Rice,supermarket,128000.00,0,1,2018-07
1,9446359,31885876,2018-07-15 16:17:00,Rice,Rice,minimarket,102750.00,0,1,2018-07
2,9470290,31930241,2018-07-15 12:12:00,Rice,Rice,supermarket,64000.00,0,3,2018-07
3,9643416,32418582,2018-07-24 08:27:00,Rice,Rice,minimarket,65000.00,0,1,2018-07
4,9692093,32561236,2018-07-26 11:28:00,Rice,Rice,supermarket,124500.00,0,1,2018-07


2.using `.astype()`

In [63]:
df = pd.read_csv("data_input/household.csv")
df.purchase_time = df.purchase_time.astype('datetime64')

df.dtypes

receipt_id                   int64
receipts_item_id             int64
purchase_time       datetime64[ns]
category                    object
sub_category                object
format                      object
unit_price                 float64
discount                     int64
quantity                     int64
yearmonth                   object
dtype: object

3. using `pd.to_datetime()` --> referred method

In [64]:
date_ex = pd.Series(['2020-24-10'])

# will throw error:
#date_ex.astype('datetime64')

pd.to_datetime(date_ex, dayfirst=True)

0   2020-10-24
dtype: datetime64[ns]

---

Notice that all columns are in the right data types, except for `purchase_time`. The correct data type for this column would have to be a `datetime`.

To convert a column `x` to a datetime, we would use:

    `x = pd.to_datetime(x)`
    

In [65]:
household['purchase_time'] = pd.to_datetime(household['purchase_time'])
household.head()

,receipt_id,receipts_item_id,purchase_time,category,sub_category,format,unit_price,discount,quantity,yearmonth
0,9622257,32369294,2018-07-22 21:19:00,Rice,Rice,supermarket,128000.00,0,1,2018-07
1,9446359,31885876,2018-07-15 16:17:00,Rice,Rice,minimarket,102750.00,0,1,2018-07
2,9470290,31930241,2018-07-15 12:12:00,Rice,Rice,supermarket,64000.00,0,3,2018-07
3,9643416,32418582,2018-07-24 08:27:00,Rice,Rice,minimarket,65000.00,0,1,2018-07
4,9692093,32561236,2018-07-26 11:28:00,Rice,Rice,supermarket,124500.00,0,1,2018-07


As you can see from the code above,`pd.to_datetime()` could do the conversion to datetime in a smart way without datetime format string required. Convenient for sure, but for some situation, this manner of `pandas` can be a little tricky.

Suppose we have a column which stores a daily sales data from end of January to the beginning of February:

In [66]:
date = pd.Series(['30-01-2020', '31-01-2020', '01-02-2020','02-02-2020'])
date

0    30-01-2020
1    31-01-2020
2    01-02-2020
3    02-02-2020
dtype: object

The legal and cultural expectations for datetime format may vary between countries. In Indonesia for example, most people are used to storing dates in DMY order. Why it matters? Let's see what happen next when we convert our `date` to datetime object:

In [67]:
pd.to_datetime(date)

0   2020-01-30
1   2020-01-31
2   2020-01-02
3   2020-02-02
dtype: datetime64[ns]

Take a look on the third observation; `pd.to_datetime` converts it to 2nd January while the actual data represents February 2nd. The function may find the string pattern automatically and smartly, but note that for dates with multiple representations, it will infer it as a month first order by default.

That's why it's important to know that `pd.to_datetime` accepts other parameters, `format` and `dayfirst`:

In [68]:
# pd.to_datetime(date, format='%d-%m-%Y')

pd.to_datetime(date, dayfirst=True)

0   2020-01-30
1   2020-01-31
2   2020-02-01
3   2020-02-02
dtype: datetime64[ns]

In [69]:
# Solution 1
pd.to_datetime(date, format="%d-%m-%Y")


# Solution 2
pd.to_datetime(date, dayfirst=True)

0   2020-01-30
1   2020-01-31
2   2020-02-01
3   2020-02-02
dtype: datetime64[ns]

In [70]:
date_2 = pd.Series(['30/01/2020 08-00', '31/01/2020 09-00'])

pd.to_datetime(date_2, format='%d/%m/%Y %H-%M')

0   2020-01-30 08:00:00
1   2020-01-31 09:00:00
dtype: datetime64[ns]

In [71]:
date_3 = pd.Series(['30/01/2020 08-00', '01/29/2020 09-00'])

pd.to_datetime(date_3)

0   2020-01-30 08:00:00+00:00
1   2020-01-29 09:00:00+00:00
dtype: datetime64[ns, UTC]

Using Python's `datetime` module, `pandas` pass the date string to `.strptime()` and follows by what's called Python's strptime directives. The full list of directives can be found in this [Documentation](https://strftime.org/).

In [72]:
household.dtypes

receipt_id                   int64
receipts_item_id             int64
purchase_time       datetime64[ns]
category                    object
sub_category                object
format                      object
unit_price                 float64
discount                     int64
quantity                     int64
yearmonth                   object
dtype: object

Other than `to_datetime`, `pandas` has a number of machineries to work with `datetime` objects. These are convenient for when we need to extract the `month`, or `year`, or `weekday_name` from `datetime`. Some common applications in business analysis include:

- `household['purchase_time'].dt.month`
- `household['purchase_time'].dt.month_name()`
- `household['purchase_time'].dt.year`
- `household['purchase_time'].dt.day` --> 1-31
- `household['purchase_time'].dt.dayofweek` --> 1-7
- `household['purchase_time'].dt.hour`
- `household['purchase_time'].dt.weekday_name` *

**Update on pandas v.1.0:**
- \* `.dt.weekday_name` can no longer can be used. To extract day names of the DateTimeIndex use `.dt.day_name()` instead

There are also other functions that can be helpful in certain situations. Supposed we want to transform the existing `datetime` column into values of periods we can use the `.to_period` method:

- `household['purchase_time'].dt.to_period('D')`
- `household['purchase_time'].dt.to_period('W')`
- `household['purchase_time'].dt.to_period('M')`
- `household['purchase_time'].dt.to_period('Q')`

In [73]:
household.purchase_time.dt.month
household.purchase_time.dt.to_period('M')

0        2018-07
1        2018-07
2        2018-07
3        2018-07
4        2018-07
          ...   
71995    2017-12
71996    2017-12
71997    2017-12
71998    2017-12
71999    2017-12
Name: purchase_time, Length: 72000, dtype: period[M]

**Knowledge Check:** Date time types  
_Est. Time required: 20 minutes_

1. In the following cell, start again by reading in the `household.csv` dataset. Drop `receipt_id` and `sub_category` columns as we won't use the columns for our analysis.  
2. Make sure the `purchase_time` column has converted as a datetime object.
3. Use `x.dt.weekday_name`/`x.dt.day_name()`, assuming `x` is a datetime object to get the day of week. Assign this to a new column in your `household` Data Frame, name it `weekday`
4. The `yearmonth` column stores the information of year and month of the `purchase_time`. Using `dt.to_period()`, how will you recreate the column if you needed the same information?
5. Print the first 5 rows of your data to verify that your preprocessing steps are correct

In [85]:
household_2 = pd.read_csv('data_input/household.csv')
household_2 = household_2.drop(['receipt_id','sub_category'],axis=1)
household_2['purchase_time'] = pd.to_datetime(household_2['purchase_time'])

household_2['weekday'] = household_2['purchase_time'].dt.day_name()

household_2.yearmonth = household_2.purchase_time.dt.to_period('M')
household_2.head()

,receipts_item_id,purchase_time,category,format,unit_price,discount,quantity,yearmonth,weekday
0,32369294,2018-07-22 21:19:00,Rice,supermarket,128000.00,0,1,2018-07,Sunday
1,31885876,2018-07-15 16:17:00,Rice,minimarket,102750.00,0,1,2018-07,Sunday
2,31930241,2018-07-15 12:12:00,Rice,supermarket,64000.00,0,3,2018-07,Sunday
3,32418582,2018-07-24 08:27:00,Rice,minimarket,65000.00,0,1,2018-07,Tuesday
4,32561236,2018-07-26 11:28:00,Rice,supermarket,124500.00,0,1,2018-07,Thursday


In [75]:
## Your code below


## -- Solution code

Tips: In the cell above, start from:

`household = pd.read_csv("data_input/household.csv")`

Inspect the first 5 rows of your data and pay close attention to the `weekday` column. 

*Bonus challenge:*  

Suppose that the estimated shipping time will take around 2 days after the products being purchased. Create a new column, name it `shipdate_est` which stores the estimated shipping time of each transaction!

In [76]:
## Your code below
# 1
household['shipdate_est'] = household['purchase_time'].dt.to_period('d') + 2

# 2
household['shipdate_est'] = household['purchase_time'] + pd.Timedelta(days = 2)

household.head()

,receipt_id,receipts_item_id,purchase_time,category,sub_category,format,unit_price,discount,quantity,yearmonth,shipdate_est
0,9622257,32369294,2018-07-22 21:19:00,Rice,Rice,supermarket,128000.00,0,1,2018-07,2018-07-24 21:19:00
1,9446359,31885876,2018-07-15 16:17:00,Rice,Rice,minimarket,102750.00,0,1,2018-07,2018-07-17 16:17:00
2,9470290,31930241,2018-07-15 12:12:00,Rice,Rice,supermarket,64000.00,0,3,2018-07,2018-07-17 12:12:00
3,9643416,32418582,2018-07-24 08:27:00,Rice,Rice,minimarket,65000.00,0,1,2018-07,2018-07-26 08:27:00
4,9692093,32561236,2018-07-26 11:28:00,Rice,Rice,supermarket,124500.00,0,1,2018-07,2018-07-28 11:28:00


More on `Timedelta`.

Use `pd.Timedelta()` to calculate difference between two dates or times:

In [77]:
t1 = pd.to_datetime('1/1/2020 01:00')
t2 = pd.to_datetime('1/1/2020 03:00')

t2 - t1

Timedelta('0 days 02:00:00')

In [78]:
t1 + pd.Timedelta(weeks = 1)

Timestamp('2020-01-08 01:00:00')

If you've managed the above exercises, well done! Run the following cell anyway to make sure we're at the same starting point as we go into the next chapter of working with categorical data (factors). 

In [79]:
# Reference answer
# 1-2
household = pd.read_csv("data_input/household.csv", parse_dates=['purchase_time'])
household.drop(['receipt_id', 'sub_category'], axis=1, inplace=True)
# 3
household['weekday'] = household['purchase_time'].dt.day_name()
# 4
household['yearmonth'] = household['purchase_time'].dt.to_period('M')
# 5
household['shipdate_est'] = household['purchase_time'] + pd.Timedelta(days = 2)

household.head()

,receipts_item_id,purchase_time,category,format,unit_price,discount,quantity,yearmonth,weekday,shipdate_est
0,32369294,2018-07-22 21:19:00,Rice,supermarket,128000.00,0,1,2018-07,Sunday,2018-07-24 21:19:00
1,31885876,2018-07-15 16:17:00,Rice,minimarket,102750.00,0,1,2018-07,Sunday,2018-07-17 16:17:00
2,31930241,2018-07-15 12:12:00,Rice,supermarket,64000.00,0,3,2018-07,Sunday,2018-07-17 12:12:00
3,32418582,2018-07-24 08:27:00,Rice,minimarket,65000.00,0,1,2018-07,Tuesday,2018-07-26 08:27:00
4,32561236,2018-07-26 11:28:00,Rice,supermarket,124500.00,0,1,2018-07,Thursday,2018-07-28 11:28:00


More on `inplace=True`:

In [80]:
# no need to assign with `=` --> `inplace=True`
household = pd.read_csv("data_input/household.csv", parse_dates=['purchase_time'])
household.drop(['receipt_id','sub_category'],axis=1, inplace=True)

## Working with Categories

From the output of `dtypes`, we see that there are three variables currently stored as `object` type where a `category` is more appropriate. This is a common diagnostic step, and one that you will employ in almost every data analysis project.

In [81]:
household.dtypes

receipts_item_id             int64
purchase_time       datetime64[ns]
category                    object
format                      object
unit_price                 float64
discount                     int64
quantity                     int64
yearmonth                   object
dtype: object

We'll convert the `weekday` column to a categorical type using `.astype()`. `astype('int64')` converts a Series to an integer type, and `.astype(category)` logically, converts a Series to a categorical.

By default, `.astype()` will raise an error if the conversion is not successful (we call them "exceptions"). In an analysis-driven environment, this is what we usually prefer. However, in certain production settings, you don't want the exception to be raised and rather return the original object (`errors='ignore'`).

In [84]:
household['weekday'] = household['weekday'].astype('category', errors='ignore')
household.dtypes

KeyError: 'weekday'

In [ ]:
household[['category','format','weekday']] = household[['category','format','weekday']].astype('category')
household.dtypes

In [ ]:
household['format'].unique()

In [ ]:
household['format'].cat.categories

Go ahead and perform the other conversions in the following cell. When you're done, use `dtypes` to check that you have the categorical columns stored as `category`.

### Alternative Solutions (optional)

In [ ]:
household = pd.read_csv('data_input/household.csv', parse_dates=['purchase_time'])
household.dtypes

#### Solution 1:

In [ ]:
household.select_dtypes(exclude='object')

In [ ]:
pd.concat([
    household.select_dtypes(exclude='object'),
    household.select_dtypes(include='object').apply(
        pd.Series.astype, dtype='category'
    )
], axis=1).dtypes

---

In [124]:
df_default = household.select_dtypes(exclude='object')
df_new = household.select_dtypes(include='object').apply(pd.Series.astype, dtype='category')

df = pd.concat([df_default, df_new], axis=1)
df.dtypes

receipt_id                   int64
receipts_item_id             int64
purchase_time       datetime64[ns]
unit_price                 float64
discount                     int64
quantity                     int64
sales_price                float64
category                  category
sub_category              category
format                    category
yearmonth                 category
day_name                  category
dtype: object

**Inclass Question**:

- Rearrange columns to original position:

In [ ]:
column_names = household.columns

df = df.reindex(columns=column_names)
df.dtypes

#### Solution 2

In [ ]:
objectcols = household.select_dtypes(include='object')
household[objectcols.columns] = objectcols.apply(lambda x: x.astype('category'))
household.dtypes

# Contingency Tables

One of the simplest EDA toolkit is the frequency table (contingency tables) and cross-tabulation tables. It is highly familiar, convenient, and practical for a wide array of statistical tasks. The simplest form of a table is to display counts of a `categorical` column. Let's start by reading our dataset in; Create a new cell and peek at the first few rows of the data.

In [4]:
household = pd.read_csv("data_input/household.csv", parse_dates=['purchase_time'])
household.head()

,receipt_id,receipts_item_id,purchase_time,category,sub_category,format,unit_price,discount,quantity,yearmonth
0,9622257,32369294,2018-07-22 21:19:00,Rice,Rice,supermarket,128000.0,0,1,2018-07
1,9446359,31885876,2018-07-15 16:17:00,Rice,Rice,minimarket,102750.0,0,1,2018-07
2,9470290,31930241,2018-07-15 12:12:00,Rice,Rice,supermarket,64000.0,0,3,2018-07
3,9643416,32418582,2018-07-24 08:27:00,Rice,Rice,minimarket,65000.0,0,1,2018-07
4,9692093,32561236,2018-07-26 11:28:00,Rice,Rice,supermarket,124500.0,0,1,2018-07


In `pandas`, each column of a `DataFrame` is a `Series`. To get the counts of each unique levels in a categorical column, we can use `.value_counts()`. The resulting object is a `Series` and in descending order so that the most frequent element is on top. 

Try and perform `.value_counts()` on the `format` column, adding either:

- `sort=False` as a parameter to prevent any sorting of elements, or
- `ascending=True` as a parameter to sort in ascending order instead

In [87]:
household.sub_category.value_counts(sort=False, ascending=True)

Rice         12000
Detergent    36000
Sugar        24000
Name: sub_category, dtype: int64

In [ ]:
## Your code below


## -- Solution code

`crosstab` is a very versatile solution to producing frequency tables on a `DataFrame` object. Its utility really goes further than that but we'll start with a simple use-case.

Consider the following code: we use `pd.crosstab()` passing in the values to group by in the rows (`index`) and columns (`columns`) respectively. 

In [88]:
pd.crosstab(index = household.sub_category, columns='total')

col_0,total
sub_category,
Detergent,36000
Rice,12000
Sugar,24000


In [89]:
pd.crosstab(index=household['sub_category'], columns="count")

col_0,count
sub_category,
Detergent,36000
Rice,12000
Sugar,24000


Realize that in the code above, we're setting the row (index) to be `sub_category` and the function will by default compute a frequency table. 

In [90]:
pd.crosstab(index=household['sub_category'], columns="count", normalize='all')

col_0,count
sub_category,
Detergent,0.50
Rice,0.17
Sugar,0.33


In the cell above, we set the values to be normalized over each columns, and this will divide each values in place over the sum of all values. This is equivalent to a manual calculation:

In [91]:
catego = pd.crosstab(index=household['sub_category'], columns="count")

catego / catego.sum()
#catego / catego.sum()

col_0,count
sub_category,
Detergent,0.50
Rice,0.17
Sugar,0.33


In [92]:
pd.crosstab(index = household['sub_category'], columns= household['format'], normalize='all')*100

format,hypermarket,minimarket,supermarket
sub_category,,,
Detergent,3.63,33.81,12.56
Rice,1.39,9.84,5.43
Sugar,2.45,21.35,9.54


In [93]:
pd.crosstab(index = household['sub_category'], columns= household['format'], normalize='index')*100

format,hypermarket,minimarket,supermarket
sub_category,,,
Detergent,7.25,67.62,25.12
Rice,8.33,59.07,32.61
Sugar,7.34,64.04,28.62


In [94]:
pd.crosstab(index = household['sub_category'], columns= household['format'], normalize='columns')*100

format,hypermarket,minimarket,supermarket
sub_category,,,
Detergent,48.61,52.02,45.62
Rice,18.60,15.14,19.74
Sugar,32.79,32.84,34.65


We can also use the same `crosstab` method to compute a cross-tabulation of two factors. In the following cell, the `index` references the sub-category column while the `columns` references the format column:

In [95]:
pd.crosstab(index=household['sub_category'], columns=household['format'])

format,hypermarket,minimarket,supermarket
sub_category,,,
Detergent,2611,24345,9044
Rice,999,7088,3913
Sugar,1761,15370,6869


This is intuitive in a way: We use `crosstab()` which, we recall, computes the count and we pass in `index` and `columns` which correspond to the row and column respectively.

When we add `margins=True` to our method call, then an extra row and column of margins (subtotals) will be included in the output:

In [96]:
pd.crosstab(index=household['sub_category'], 
            columns=household['format'], 
            normalize='columns',
            margins=True)*100

format,hypermarket,minimarket,supermarket,All
sub_category,,,,
Detergent,48.61,52.02,45.62,50.00
Rice,18.60,15.14,19.74,16.67
Sugar,32.79,32.84,34.65,33.33


**Exercise**: (15 - 20 minutes): until 19:30

In the following cell, use `pd.crosstab()` with `yearmonth` as the row and `format` as the column. Set `margins=True` to get a total across the row and columns. 

1. Say, we had special promotions in February 2018 and need to observe the transaction counts during that period. Use any subsetting method you've learned to get only sales data in February 2018!

2. In which period (yearmonth) did hypermarket has its biggest total transactions?



In [ ]:
q1 = pd.crosstab(index=household['yearmonth'], columns=household['format'],  margins=True)
q1.loc[['2018-02']]

In [ ]:
cross_year_hypermarket =  pd.crosstab(index=household['yearmonth'],columns=household['format'])
cond1 = cross_year_hypermarket['hypermarket'] == cross_year_hypermarket['hypermarket'].max()

cross_year_hypermarket[cond1].drop(columns = ['minimarket', 'supermarket'])

In [ ]:
cross_year_hypermarket = pd.crosstab(index=household['yearmonth'],columns=household['format'])

cross_year_hypermarket[cross_year_hypermarket['hypermarket']==cross_year_hypermarket['hypermarket'].max()].\
drop(columns=['minimarket','supermarket']) 

In [ ]:
# `.idxmax()` --> index dari nilai tertinggi
df = pd.crosstab(index=household['yearmonth'],columns=household['format'])
df.loc[[df.hypermarket.idxmax()], ['hypermarket']]

In [ ]:
df.sort_values('hypermarket', ascending= False).head(1)

Suppose you want to compare the total transactions occured in each markets (`format`) by the year quarter. Create a new column, name it `quarter`, to get the quarter period information from `purchase_time`, then use `pd.crosstab()` to create a frequency table which shown the total transactions per each quarter

3. In 2018, does the minimarket reach its highest transaction in Q2? 
4. What about supermarket? In which quarter period did it reach its highest total transaction?

In [ ]:
household = pd.read_csv('data_input/household.csv')

household['purchase_time'] = pd.to_datetime(household['purchase_time'])
household['quarter'] = household['purchase_time'].dt.to_period('Q') 

household.head()

In [ ]:
pd.crosstab(index=household['quarter'],columns=household['format'], margins=True)*100 

In [ ]:
pd.crosstab(index = household['format'], columns=household['quarter'])

If you want an extra challenge, try and modify your code above to include a `normalize` parameter. 

`normalize` accepts a boolean value, or one of `all`, `index` or `columns`. Since we want it to normalize across each row, we will set this parameter to the value of `index`.

## Aggregation Table

In the following section, we will introduce another parameter to perform aggregation on our table. The `aggfunc` parameter when present, required the `values` parameter to be specified as well. `values` is the values to aggregate according to the factors in our index and columns:

In [ ]:
pd.crosstab(index = household['sub_category'], # categorical
            columns = 'rata-rata',
            values = household['unit_price'], #numeric,
            aggfunc = 'mean')

In [ ]:
pd.crosstab(index = household['sub_category'], # categorical
            columns = household['format'],
            values = household['unit_price'], #numeric,
            aggfunc = 'mean').round(2)

In [ ]:
pd.crosstab(index=household['sub_category'], 
            columns='mean', 
            values=household['unit_price'],
            aggfunc='mean')

**Knowledge Check**: Cross tabulation  

Create a cross-tab using `sub_category` as the index (row) and `format` as the column. Fill the values with the median of `unit_price` across each row and column. Add a subtotal to both the row and column by setting `margins=True`.

1. On average, Sugar is cheapest at...?
2. On average, Detergent is most expensive at...?

Create a new cell for your code and answer the questions above.

In [ ]:
pd.crosstab(index=household['sub_category'],
           columns=household['format'],
           values=household['unit_price'],
           aggfunc='sum', margins=True)

In [ ]:
## Your code below


## -- Solution code

Reference answer:

```
pd.crosstab(index=household['sub_category'], 
            columns=household['format'], 
            values=household['unit_price'],
            aggfunc='median', margins=True)
```

**Exercise**: (15-20 minutes)

The company is planning on create a promotional campaign for each Rice, Detergent and Sugar products. They need to decide whether the promotion is better be held on Weekdays or Weekend as they prefer to have the promotions in the times they have the least revenue.

1. Suppose you were asked to analyse the revenue from the recorded sales price. In average, does the sales in Weekend actually higher than Weekdays? Using `pd.crosstab()`, create an aggregation table which could explain your justification!


2. Between Detergent, Rice, and Sugar, which category sold in the highest total quantity in each day period?

In [119]:
# 1
household["day_name"] = household["purchase_time"].dt.day_name()
household["sales_price"] = household["unit_price"] * household["quantity"] 

pd.crosstab(
    index= household['day_name'],
    columns= 'mean',
    values= household['sales_price'],
    aggfunc= 'mean'
).round(2)

col_0,mean
day_name,
Friday,34480.24
Monday,33819.61
Saturday,34497.41
Sunday,34999.88
Thursday,34298.07
Tuesday,33839.78
Wednesday,33040.81


---
**In class question**:
1. Ordering `household.day_name` by actual day order

**Solution 1**:
- Reindexing the crosstab

In [120]:
cross = pd.crosstab(index= household['day_name'],
                    columns= 'mean',
                    values= household['sales_price'],
                    aggfunc= 'mean').round(2)
cross

col_0,mean
day_name,
Friday,34480.24
Monday,33819.61
Saturday,34497.41
Sunday,34999.88
Thursday,34298.07
Tuesday,33839.78
Wednesday,33040.81


In [99]:
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
cross.reindex(day_order)

col_0,mean
day_name,
Monday,33819.61
Tuesday,33839.78
Wednesday,33040.81
Thursday,34298.07
Friday,34480.24
Saturday,34497.41
Sunday,34999.88


**Solution 2**:
- Ordering categorical column (preparation phase)

In [133]:
household["day_name"] = household["purchase_time"].dt.day_name()
household['day_name'] = household['day_name'].astype('category')

household['day_name'].unique()

[Sunday, Tuesday, Thursday, Wednesday, Saturday, Monday, Friday]
Categories (7, object): [Sunday, Tuesday, Thursday, Wednesday, Saturday, Monday, Friday]

In [135]:
# list of category in ordered values
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

household['day_name'] = pd.Categorical(household['day_name'], categories=day_order, ordered=True)
household['day_name'].unique()

[Sunday, Tuesday, Thursday, Wednesday, Saturday, Monday, Friday]
Categories (7, object): [Monday < Tuesday < Wednesday < Thursday < Friday < Saturday < Sunday]

In [136]:
pd.crosstab(
    index= household['day_name'],
    columns= 'mean',
    values= household['sales_price'],
    aggfunc= 'mean'
).round(2)

col_0,mean
day_name,
Monday,33819.61
Tuesday,33839.78
Wednesday,33040.81
Thursday,34298.07
Friday,34480.24
Saturday,34497.41
Sunday,34999.88


**In class question**:
2. Add new column for Weekend/Weekdays information (create new column based on condition)

In [137]:
import numpy as np

In [103]:
# np.select(condlist, choicelist, default)

np.select()

cond = household['day_name'].isin(['Saturday', 'Sunday'])
household['is_weekend'] = np.select([cond], ['Weekend'], default = 'Weekdays')

household.head()

,receipt_id,receipts_item_id,purchase_time,category,sub_category,format,unit_price,discount,quantity,yearmonth,day_name,sales_price,is_weekend
0,9622257,32369294,2018-07-22 21:19:00,Rice,Rice,supermarket,128000.00,0,1,2018-07,Sunday,128000.00,Weekend
1,9446359,31885876,2018-07-15 16:17:00,Rice,Rice,minimarket,102750.00,0,1,2018-07,Sunday,102750.00,Weekend
2,9470290,31930241,2018-07-15 12:12:00,Rice,Rice,supermarket,64000.00,0,3,2018-07,Sunday,192000.00,Weekend
3,9643416,32418582,2018-07-24 08:27:00,Rice,Rice,minimarket,65000.00,0,1,2018-07,Tuesday,65000.00,Weekdays
4,9692093,32561236,2018-07-26 11:28:00,Rice,Rice,supermarket,124500.00,0,1,2018-07,Thursday,124500.00,Weekdays


In [151]:
# option 1
cond1 = household['day_name'] == 'Saturday'
cond2 = household['day_name'] == 'Sunday'
cond1 | cond2

# option 2
household['day_name'].isin(['Saturday', 'Sunday'])

0         True
1         True
2         True
3        False
4        False
         ...  
71995    False
71996    False
71997    False
71998    False
71999    False
Name: day_name, Length: 72000, dtype: bool

In [146]:
cond = household['day_name'].isin(['Saturday', 'Sunday'])

household['is_weekend'] = np.select([cond], ['Weekend'], default = 'Weekdays')
household.head()

,receipt_id,receipts_item_id,purchase_time,category,sub_category,format,unit_price,discount,quantity,yearmonth,day_name,sales_price,is_weekend
0,9622257,32369294,2018-07-22 21:19:00,Rice,Rice,supermarket,128000.00,0,1,2018-07,Sunday,128000.00,Weekend
1,9446359,31885876,2018-07-15 16:17:00,Rice,Rice,minimarket,102750.00,0,1,2018-07,Sunday,102750.00,Weekend
2,9470290,31930241,2018-07-15 12:12:00,Rice,Rice,supermarket,64000.00,0,3,2018-07,Sunday,192000.00,Weekend
3,9643416,32418582,2018-07-24 08:27:00,Rice,Rice,minimarket,65000.00,0,1,2018-07,Tuesday,65000.00,Weekdays
4,9692093,32561236,2018-07-26 11:28:00,Rice,Rice,supermarket,124500.00,0,1,2018-07,Thursday,124500.00,Weekdays


In [ ]:
#df.rename(columns={"A": "a", "B": "c"})

# household.rename(columns = {'receipt_id' : 'ID'})

---

In [155]:
# 2
pd.crosstab(
    index= household.sub_category,
    columns = household.day_name,
    values= household['quantity'],
    aggfunc= 'sum'
).round(2)

day_name,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
sub_category,,,,,,,
Detergent,5845,6193,6106,5923,8216,8626,8751
Rice,2037,2078,1949,2068,2273,2581,3009
Sugar,5427,5527,5580,5627,5940,6506,6504


In [156]:
# or:
pd.crosstab(
    index= [household.day_name, household.sub_category],
    columns = 'Items Sold',
    values= household['quantity'],
    aggfunc= 'sum'
)

col_0                   Items Sold
day_name  sub_category            
Monday    Detergent           5845
          Rice                2037
          Sugar               5427
Tuesday   Detergent           6193
          Rice                2078
          Sugar               5527
Wednesday Detergent           6106
          Rice                1949
          Sugar               5580
Thursday  Detergent           5923
          Rice                2068
          Sugar               5627
Friday    Detergent           8216
          Rice                2273
          Sugar               5940
Saturday  Detergent           8626
          Rice                2581
          Sugar               6506
Sunday    Detergent           8751
          Rice                3009
          Sugar               6504

### Higher-dimensional Tables

If we need to inspect our data in higher resolution, we can create cross-tabulation using more than one factor. This allows us to yield insights on a more granular level yet have our output remain relatively compact and structured:

In [157]:
pd.crosstab(index=household['yearmonth'], 
            columns=[household['format'], household['sub_category']], 
            values=household['unit_price'],
            aggfunc='median')

format       hypermarket                   minimarket                    \
sub_category   Detergent     Rice    Sugar  Detergent     Rice    Sugar   
yearmonth                                                                 
2017-10         17400.00 64000.00 12500.00   16800.00 62500.00 12500.00   
2017-11         16770.00 64000.00 12400.00   16800.00 62500.00 12500.00   
2017-12         17500.00 64000.00 12000.00   16600.00 62500.00 12500.00   
2018-01         16800.00 64000.00 12275.00   16200.00 62500.00 12500.00   
2018-02         17500.00 64000.00 11990.00   17000.00 63500.00 12500.00   
2018-03         16900.00 64000.00 12000.00   16300.00 63500.00 12500.00   
2018-04         16815.00 64000.00 11990.00   16800.00 63500.00 12500.00   
2018-05         16950.00 64000.00 12000.00   16800.00 63000.00 12500.00   
2018-06         16550.00 64000.00 12300.00   17300.00 63500.00 12500.00   
2018-07         16550.00 64000.00 12325.00   16800.00 63500.00 12500.00   
2018-08         16839.00 62600.00 12000.00   17500.00 62500.00 12500.00   
2018-09         16720.00 60000.00 11900.00   16900.00 62500.00 12500.00   

format       supermarket                    
sub_category   Detergent     Rice    Sugar  
yearmonth                                   
2017-10         16925.00 64000.00 12500.00  
2017-11         16500.00 64000.00 12400.00  
2017-12         16600.00 64000.00 12400.00  
2018-01         16700.00 64000.00 12400.00  
2018-02         16200.00 64000.00 12290.00  
2018-03         15680.00 64000.00 12400.00  
2018-04         15700.00 64000.00 12400.00  
2018-05         16700.00 64000.00 12400.00  
2018-06         16700.00 64000.00 12400.00  
2018-07         16600.00 64000.00 12300.00  
2018-08         17000.00 64000.00 12300.00  
2018-09         16990.00 62550.00 12300.00

In `pandas` we call a higher-dimensional tables as Multi-Index Dataframe. We are going to dive deeper into the structure of the object on the the next chapter.

## Pivot Tables

If our data is already in a `DataFrame` format, using `pd.pivot_table` can sometimes be more convenient compared to a `pd.crosstab`. 

Fortunately, much of the parameters in a `pivot_table()` function is the same as `pd.crosstab()`. The noticable difference is the use of an additional `data` parameter, which allow us to specify the `DataFrame` that is used to construct the pivot table.

We create a `pivot_table` by passing in the following:
- `data`: our `DataFrame`
- `index`: the column to be used as rows
- `columns`: the column to be used as columns
- `values`: the values used to fill in the table
- `aggfunc`: the aggregation function

In [ ]:
pd.crosstab(index=household['yearmonth'], 
            columns=[household['format'], household['sub_category']], 
            values=household['unit_price'],
            aggfunc='median')

In [164]:
pd.pivot_table(
    data=household,
    index= 'yearmonth',
    columns= ['format', 'sub_category'],
    values= 'unit_price',
    aggfunc= 'median'
)

format       hypermarket                   minimarket                    \
sub_category   Detergent     Rice    Sugar  Detergent     Rice    Sugar   
yearmonth                                                                 
2017-10         17400.00 64000.00 12500.00   16800.00 62500.00 12500.00   
2017-11         16770.00 64000.00 12400.00   16800.00 62500.00 12500.00   
2017-12         17500.00 64000.00 12000.00   16600.00 62500.00 12500.00   
2018-01         16800.00 64000.00 12275.00   16200.00 62500.00 12500.00   
2018-02         17500.00 64000.00 11990.00   17000.00 63500.00 12500.00   
2018-03         16900.00 64000.00 12000.00   16300.00 63500.00 12500.00   
2018-04         16815.00 64000.00 11990.00   16800.00 63500.00 12500.00   
2018-05         16950.00 64000.00 12000.00   16800.00 63000.00 12500.00   
2018-06         16550.00 64000.00 12300.00   17300.00 63500.00 12500.00   
2018-07         16550.00 64000.00 12325.00   16800.00 63500.00 12500.00   
2018-08         16839.00 62600.00 12000.00   17500.00 62500.00 12500.00   
2018-09         16720.00 60000.00 11900.00   16900.00 62500.00 12500.00   

format       supermarket                    
sub_category   Detergent     Rice    Sugar  
yearmonth                                   
2017-10         16925.00 64000.00 12500.00  
2017-11         16500.00 64000.00 12400.00  
2017-12         16600.00 64000.00 12400.00  
2018-01         16700.00 64000.00 12400.00  
2018-02         16200.00 64000.00 12290.00  
2018-03         15680.00 64000.00 12400.00  
2018-04         15700.00 64000.00 12400.00  
2018-05         16700.00 64000.00 12400.00  
2018-06         16700.00 64000.00 12400.00  
2018-07         16600.00 64000.00 12300.00  
2018-08         17000.00 64000.00 12300.00  
2018-09         16990.00 62550.00 12300.00

Total (sum) quantity dari setiap sub_category.

In [166]:
## Your code here
pd.pivot_table(data=household,
               index='yearmonth',
               columns=['sub_category'],
               values='quantity',
               aggfunc='sum') 

sub_category,Detergent,Rice,Sugar
yearmonth,,,
2017-10,4067,1422,3171
2017-11,3877,1298,3181
2017-12,3970,1277,3300
2018-01,4241,1323,3212
2018-02,4280,1353,3391
2018-03,4619,1413,3698
2018-04,4415,1363,3582
2018-05,4153,1359,3840
2018-06,4037,1358,3895


In [160]:
pd.pivot_table(
    data=household,
    index='yearmonth',
    columns=['format','sub_category'],
    values='unit_price',
    aggfunc='median'
)

format       hypermarket                   minimarket                    \
sub_category   Detergent     Rice    Sugar  Detergent     Rice    Sugar   
yearmonth                                                                 
2017-10         17400.00 64000.00 12500.00   16800.00 62500.00 12500.00   
2017-11         16770.00 64000.00 12400.00   16800.00 62500.00 12500.00   
2017-12         17500.00 64000.00 12000.00   16600.00 62500.00 12500.00   
2018-01         16800.00 64000.00 12275.00   16200.00 62500.00 12500.00   
2018-02         17500.00 64000.00 11990.00   17000.00 63500.00 12500.00   
2018-03         16900.00 64000.00 12000.00   16300.00 63500.00 12500.00   
2018-04         16815.00 64000.00 11990.00   16800.00 63500.00 12500.00   
2018-05         16950.00 64000.00 12000.00   16800.00 63000.00 12500.00   
2018-06         16550.00 64000.00 12300.00   17300.00 63500.00 12500.00   
2018-07         16550.00 64000.00 12325.00   16800.00 63500.00 12500.00   
2018-08         16839.00 62600.00 12000.00   17500.00 62500.00 12500.00   
2018-09         16720.00 60000.00 11900.00   16900.00 62500.00 12500.00   

format       supermarket                    
sub_category   Detergent     Rice    Sugar  
yearmonth                                   
2017-10         16925.00 64000.00 12500.00  
2017-11         16500.00 64000.00 12400.00  
2017-12         16600.00 64000.00 12400.00  
2018-01         16700.00 64000.00 12400.00  
2018-02         16200.00 64000.00 12290.00  
2018-03         15680.00 64000.00 12400.00  
2018-04         15700.00 64000.00 12400.00  
2018-05         16700.00 64000.00 12400.00  
2018-06         16700.00 64000.00 12400.00  
2018-07         16600.00 64000.00 12300.00  
2018-08         17000.00 64000.00 12300.00  
2018-09         16990.00 62550.00 12300.00

A key difference between `crosstab` and `pivot_table` is that `crosstab` uses `len` (or `count`) as the default aggregation function while `pivot_table` using the mean. Copy the code from the cell above and make a change: use `sum` as the aggregation function instead: 

In [ ]:
## Your code below


## -- Solution code

---
### pandas Tables summary:

 1. **Frequency Table:** count n data


In [6]:
# simplest approach: using value_counts()

household.format.value_counts()
household.format.value_counts(normalize = True)*100

# more versatile: use `pd.crosstab(index, column)`
pd.crosstab(
    index=household.format,
    columns='Total Frequency'
)

pd.crosstab(
    index=household.format,
    columns=household.sub_category
)

sub_category,Detergent,Rice,Sugar
format,,,
hypermarket,2611,999,1761
minimarket,24345,7088,15370
supermarket,9044,3913,6869


2. **Aggregation tabble:** aggregate numerical value

In [17]:
# using `pd.crosstab(index, columns, values, aggfunc)`
pd.crosstab(
    index = household.format,
    columns = 'Total Item Sold',
    values= household.quantity,
    aggfunc= 'sum'
)

# using `pd.pivot_table(data, index &/columns, values, aggfunc)`
pd.pivot_table(
    data=household,
    index='format',
    values= 'quantity',
    aggfunc= 'sum'
)

,quantity
format,
hypermarket,8688
minimarket,67980
supermarket,30098


---

Workflow:

1. Read data
2. Explore
3. Cleaning data:
    - Missing Values
    - Duplicate
    - Data Types
    - Tidying string objects
    - ....
4. Transform/Wrangling for further analysis


# Missing Values and Duplicates

During the data exploration and preparation phase, it is likely we come across some problematic details in our data. This could be the value of _-1_ for the _age_ column, a value of _blank_ for the _customer segment_ column, or a value of _None_ for the _loan duration_ column. All of these are examples of "untidy" data, which is rather common depending on the data collection and recording process in a company.

In `pandas`, we use `NaN` (not a number) to denote missing data; The equivalent for datetime is `NaT` but both are essentially compatible with each other. From the docs:
> The choice of using `NaN` internally to denote missing data was largely for simplicity and performance reasons. We are hopeful that NumPy will soon be able to provide a native NA type solution (similar to R) performant enough to be used in pandas.

In [19]:
household = pd.read_csv("data_input/household.csv", index_col=1, parse_dates=['purchase_time'])
household.drop(['receipt_id', 'yearmonth', 'sub_category'], axis=1, inplace=True)
household['weekday'] = household['purchase_time'].dt.day_name() 
household.head()

,purchase_time,category,format,unit_price,discount,quantity,weekday
receipts_item_id,,,,,,,
32369294,2018-07-22 21:19:00,Rice,supermarket,128000.0,0,1,Sunday
31885876,2018-07-15 16:17:00,Rice,minimarket,102750.0,0,1,Sunday
31930241,2018-07-15 12:12:00,Rice,supermarket,64000.0,0,3,Sunday
32418582,2018-07-24 08:27:00,Rice,minimarket,65000.0,0,1,Tuesday
32561236,2018-07-26 11:28:00,Rice,supermarket,124500.0,0,1,Thursday


In [171]:
import math 

x = [i for i in range(32000000, 32000005)]
x.insert(2, 32030785)
x

[32000000, 32000001, 32030785, 32000002, 32000003, 32000004]

In [20]:
import math
x=[i for i in range(32000000, 32000005)]
x.insert(2,32030785)

household2 = household.head(6).copy()
household2 = household2.reindex(x)
household2 = pd.concat([household2, household.head(14)])
household2.loc[31885876, "weekday"] = math.nan
household2.iloc[2:8,]

,purchase_time,category,format,unit_price,discount,quantity,weekday
receipts_item_id,,,,,,,
32030785,2018-07-17 18:05:00,Rice,minimarket,63500.0,0.0,1.0,Tuesday
32000002,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000003,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000004,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32369294,2018-07-22 21:19:00,Rice,supermarket,128000.0,0.0,1.0,Sunday
31885876,2018-07-15 16:17:00,Rice,minimarket,102750.0,0.0,1.0,NaN


In [199]:
household2.tail()

,receipt_id,receipts_item_id,purchase_time,category,sub_category,format,unit_price,discount,quantity,yearmonth,day_name,sales_price,is_weekend,weekday
10,9198777.00,31125365.00,2018-07-02 15:39:00,Rice,Rice,minimarket,112500.00,0.00,1.00,2018-07,Monday,112500.00,Weekdays,nan
11,9763483.00,32856560.00,2018-07-31 05:51:00,Rice,Rice,hypermarket,77750.00,0.00,1.00,2018-07,Tuesday,77750.00,Weekdays,nan
12,9691090.00,32552145.00,2018-07-26 11:43:00,Rice,Rice,minimarket,66500.00,0.00,1.00,2018-07,Thursday,66500.00,Weekdays,nan
13,9624752.00,32369065.00,2018-07-23 14:22:00,Rice,Rice,minimarket,62900.00,0.00,1.00,2018-07,Monday,62900.00,Weekdays,nan
31885876,nan,nan,NaT,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,NaN,nan


In the cell above, I used `reindex` to "inject" some rows where values don't exist (receipts item id 32000000 through 32000004) and also set `math.nan` on one of the values for `weekday`. Notice from the output that between row 3 to 8 there are at least a few rows with missing data. We can use `isna()` and `notna()` to detect missing values. An example code is as below:

In [52]:
household2.isna().sum()

purchase_time    5
category         5
format           5
unit_price       5
discount         5
quantity         5
weekday          6
dtype: int64

A common way of using the `.isna()` method is to combine it with the subsetting methods we've learned in previous lessons:

In [53]:
household2[household2['weekday'].isna()]

,purchase_time,category,format,unit_price,discount,quantity,weekday
receipts_item_id,,,,,,,
32000000,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000001,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000002,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000003,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000004,NaT,NaN,NaN,NaN,NaN,NaN,NaN
31885876,2018-07-15 16:17:00,Rice,minimarket,102750.0,0.0,1.0,NaN


In [56]:
# filter missing data in any column:
household2[household2.isna().any(axis = 1)]

,purchase_time,category,format,unit_price,discount,quantity,weekday
receipts_item_id,,,,,,,
32030785,2018-07-17 18:05:00,Rice,minimarket,63500.0,0.0,1.0,Tuesday
32369294,2018-07-22 21:19:00,Rice,supermarket,128000.0,0.0,1.0,Sunday
31885876,2018-07-15 16:17:00,Rice,minimarket,102750.0,0.0,1.0,NaN
31930241,2018-07-15 12:12:00,Rice,supermarket,64000.0,0.0,3.0,Sunday
32418582,2018-07-24 08:27:00,Rice,minimarket,65000.0,0.0,1.0,Tuesday
32561236,2018-07-26 11:28:00,Rice,supermarket,124500.0,0.0,1.0,Thursday
32030785,2018-07-17 18:05:00,Rice,minimarket,63500.0,0.0,1.0,Tuesday
32935097,2018-07-29 18:18:00,Rice,supermarket,66500.0,0.0,1.0,Sunday
32593606,2018-07-25 12:48:00,Rice,minimarket,62500.0,0.0,1.0,Wednesday


Go ahead and use `notna()` to extract all the rows where `weekday` column is not missing:

In [ ]:
## Your code below


## -- Solution code

In [58]:
household2[household2.weekday.notna()]

,purchase_time,category,format,unit_price,discount,quantity,weekday
receipts_item_id,,,,,,,
32030785,2018-07-17 18:05:00,Rice,minimarket,63500.0,0.0,1.0,Tuesday
32369294,2018-07-22 21:19:00,Rice,supermarket,128000.0,0.0,1.0,Sunday
31930241,2018-07-15 12:12:00,Rice,supermarket,64000.0,0.0,3.0,Sunday
32418582,2018-07-24 08:27:00,Rice,minimarket,65000.0,0.0,1.0,Tuesday
32561236,2018-07-26 11:28:00,Rice,supermarket,124500.0,0.0,1.0,Thursday
32030785,2018-07-17 18:05:00,Rice,minimarket,63500.0,0.0,1.0,Tuesday
32935097,2018-07-29 18:18:00,Rice,supermarket,66500.0,0.0,1.0,Sunday
32593606,2018-07-25 12:48:00,Rice,minimarket,62500.0,0.0,1.0,Wednesday
32573843,2018-07-26 16:41:00,Rice,minimarket,62500.0,0.0,1.0,Thursday


Another common use-case in missing values treatment is to count the number of `NAs` across each column:

In [55]:
household2.isna().sum()

purchase_time    5
category         5
format           5
unit_price       5
discount         5
quantity         5
weekday          6
dtype: int64

When we are certain that the rows with `NA`s can be safely dropped, we can use `dropna()`, optionally specifying a threshold. By default, this method drops the row if any NA value is present (`how='any'`), but it can be set to do this only when all values are NA in that row (`how='all'`).

```
    # drops row if all values are NA
    household2.dropna(how='all')
    
    # drops row if it doesn't have at least 5 non-NA values
    household2.dropna(thresh=5) 
```

In [228]:
household2.dropna(thresh = 5) # NA in 5 columns

,purchase_time,category,format,unit_price,discount,quantity,weekday
receipts_item_id,,,,,,,
32030785,2018-07-17 18:05:00,Rice,minimarket,63500.00,0.00,1.00,Tuesday
32369294,2018-07-22 21:19:00,Rice,supermarket,128000.00,0.00,1.00,Sunday
31885876,2018-07-15 16:17:00,Rice,minimarket,102750.00,0.00,1.00,NaN
31930241,2018-07-15 12:12:00,Rice,supermarket,64000.00,0.00,3.00,Sunday
32418582,2018-07-24 08:27:00,Rice,minimarket,65000.00,0.00,1.00,Tuesday
32561236,2018-07-26 11:28:00,Rice,supermarket,124500.00,0.00,1.00,Thursday
32030785,2018-07-17 18:05:00,Rice,minimarket,63500.00,0.00,1.00,Tuesday
32935097,2018-07-29 18:18:00,Rice,supermarket,66500.00,0.00,1.00,Sunday
32593606,2018-07-25 12:48:00,Rice,minimarket,62500.00,0.00,1.00,Wednesday


When we have data where duplicated observations are recorded, we can use `.drop_duplicates()` specifying whether the first occurence or the last should be kept:

In [234]:
household2.drop_duplicates()

,purchase_time,category,format,unit_price,discount,quantity,weekday
receipts_item_id,,,,,,,
32000004,NaT,NaN,NaN,nan,nan,nan,NaN
32369294,2018-07-22 21:19:00,Rice,supermarket,128000.00,0.00,1.00,Sunday
31885876,2018-07-15 16:17:00,Rice,minimarket,102750.00,0.00,1.00,NaN
31930241,2018-07-15 12:12:00,Rice,supermarket,64000.00,0.00,3.00,Sunday
32418582,2018-07-24 08:27:00,Rice,minimarket,65000.00,0.00,1.00,Tuesday
32561236,2018-07-26 11:28:00,Rice,supermarket,124500.00,0.00,1.00,Thursday
32030785,2018-07-17 18:05:00,Rice,minimarket,63500.00,0.00,1.00,Tuesday
32935097,2018-07-29 18:18:00,Rice,supermarket,66500.00,0.00,1.00,Sunday
32593606,2018-07-25 12:48:00,Rice,minimarket,62500.00,0.00,1.00,Wednesday


In [49]:
print(household2.shape)
print(household2.drop_duplicates(keep="first").shape)

(21, 11)
(15, 11)


**Knowledge Check:** Duplicates and Missing Value  
_Est. Time required: 20 minutes_

1. Duplicates may mean a different thing from a data point-of-view and a business analyst's point-of-view. You want to be extra careful about whether the duplicates is an intended characteristic of your data, or whether it poses a violation to the business logic. 

    - a. A medical center collects anonymized heart rate monitoring data from patients. It has duplicate observations collected across a span of 3 months
    - b. An insurance company uses machine learning to deliver dynamic pricing to its customers. Each row contains the customer's name, occupation / profession and historical health data. It has duplicate observations collected across a span of 3 months
    - c. On our original `household` data, check for duplicate observations. Would you have drop the duplicated rows?

---

2. Once you've identified the missing values, there are 3 common ways to deal with it:

    - a. Use `dropna` with a reasonable threshold to remove any rows that contain too little values rendering it unhelpful to your analysis
    - b. Replace the missing values with a central value (mean or median)
    - c. Imputation through a predictive model
        - In a dataframe where `salary` is missing but the bank has data about the customer's occupation / profession, years of experience, years of education, seniority level, age, and industry, then a machine learning model such as regression or nearest neighbor can offer a viable alternative to the mean imputation approach
 
Going back to `household2`: what is a reasonable strategy? List them down or in pseudo-code.

In [237]:
Clean = household2.drop_duplicates(keep="first")
Clean = final.dropna(thresh=4)

#Clean['weekday'] = Clean['purchase_time'].dt.day_name

,purchase_time,category,format,unit_price,discount,quantity,weekday
receipts_item_id,,,,,,,
32369294,2018-07-22 21:19:00,Rice,supermarket,128000.00,0,1,Sunday
31885876,2018-07-15 16:17:00,Rice,minimarket,102750.00,0,1,Sunday
31930241,2018-07-15 12:12:00,Rice,supermarket,64000.00,0,3,Sunday
32418582,2018-07-24 08:27:00,Rice,minimarket,65000.00,0,1,Tuesday
32561236,2018-07-26 11:28:00,Rice,supermarket,124500.00,0,1,Thursday
...,...,...,...,...,...,...,...
17998610,2017-12-27 09:20:00,Sugar/Flavored Syrup,minimarket,25000.00,0,1,Wednesday
17432379,2017-12-13 19:52:00,Sugar/Flavored Syrup,minimarket,12500.00,0,1,Wednesday
18263665,2017-12-27 08:03:00,Sugar/Flavored Syrup,minimarket,12500.00,0,1,Wednesday


In [50]:
## Your code below


## -- Solution code

## Missing Values Treatment

Some common methods when working with missing values are demonstrated in the following section. We make a copy of the NA-included DataFrame, and name it `household3`:

In [78]:
household3 = household2.copy()
household3.head()

,purchase_time,category,format,unit_price,discount,quantity,weekday
receipts_item_id,,,,,,,
32000000,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000001,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32030785,2018-07-17 18:05:00,Rice,minimarket,63500.0,0.0,1.0,Tuesday
32000002,NaT,NaN,NaN,NaN,NaN,NaN,NaN
32000003,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In the following cell, the technique is demonstrably repetitive or even verbose. This is done to give us an idea of all the different options we can pick from. 

You may observe, for example that the two lines of code are functionally identical:
- `.fillna(0)`
- `.replace(np.nan, 0)`

In [89]:
# convert NA categories to 'Missing'
household3[['category', 'format','discount']] = household3[['category', 'format','discount']].fillna('Missing')

# convert NA unit_price to 0
household3.unit_price = household3.unit_price.fillna(0)

# convert NA purchase_time with 'bfill'
household3.purchase_time = pd.to_datetime(household3.purchase_time)
household3.purchase_time = household3.fillna(method='bfill')

# convert NA quantity with -1
household3.quantity = household3.quantity.fillna(-1)

household3.head()

,purchase_time,category,format,unit_price,discount,quantity,weekday
receipts_item_id,,,,,,,
32000000,2018-07-17 18:05:00,Missing,Missing,0.0,Missing,-1.0,NaN
32000001,2018-07-17 18:05:00,Missing,Missing,0.0,Missing,-1.0,NaN
32030785,2018-07-17 18:05:00,Rice,minimarket,63500.0,0,1.0,Tuesday
32000002,2018-07-22 21:19:00,Missing,Missing,0.0,Missing,-1.0,NaN
32000003,2018-07-22 21:19:00,Missing,Missing,0.0,Missing,-1.0,NaN
